# Logistics Delivery Delay Analysis

This notebook explores a synthetic logistics dataset and builds predictive models to determine whether a shipment will arrive late. The analysis includes descriptive statistics, visualizations, feature engineering, model training, and insights for business stakeholders.

Feel free to follow the sections below and adapt the code to your own datasets or business problems.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Display settings
pd.set_option('display.max_columns', None)

# Load the dataset
file_path = 'logistics_delay_data.csv'
df = pd.read_csv(file_path, parse_dates=['order_date', 'expected_delivery_date', 'actual_delivery_date'])

# Preview the data
df.head()


## Dataset Overview

We begin by examining the basic structure of the dataset: number of rows, columns, data types, and summary statistics.


In [ ]:

# Dataset shape and data types
print('Dataset shape:', df.shape)
df.dtypes


In [ ]:

# Summary statistics for numeric features
numeric_cols = ['weight_kg', 'distance_km', 'shipping_cost', 'delay_days']
df[numeric_cols].describe()


## Visualizing Delivery Delays

Let's explore how delays are distributed across shipments and whether certain attributes are associated with higher delay rates.


In [ ]:

# Histogram of delay days
plt.figure(figsize=(8, 4))
sns.histplot(df['delay_days'], bins=15, kde=True, color='steelblue')
plt.title('Distribution of Delay Days')
plt.xlabel('Delay Days')
plt.ylabel('Number of Shipments')
plt.show()

# Delay rates by shipping mode
delay_by_mode = df.groupby('shipping_mode')['is_delayed'].mean().sort_values(ascending=False)

plt.figure(figsize=(8, 4))
delay_by_mode.plot(kind='bar', color='orange')
plt.title('Average Delay Rate by Shipping Mode')
plt.ylabel('Fraction of Shipments Delayed')
plt.xlabel('Shipping Mode')
plt.show()


## Correlation Analysis

We compute correlations among numeric variables to understand relationships such as weight versus cost or distance versus delay days.


In [ ]:

# Correlation matrix
corr = df[['weight_kg', 'distance_km', 'shipping_cost', 'delay_days']].corr()

plt.figure(figsize=(6, 4))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numeric Features')
plt.show()


## Predictive Modeling

We build two models—a logistic regression and a random forest classifier—to predict whether a shipment will be delayed. We use categorical features and continuous features, encoding categories appropriately.


In [ ]:

# Feature selection
feature_cols = ['shipping_mode', 'product_category', 'product_subcategory', 'weight_kg', 'distance_km', 'warehouse_region', 'customer_segment']
X = df[feature_cols]
y = df['is_delayed']

# Define categorical and numeric columns
categorical_cols = ['shipping_mode', 'product_category', 'product_subcategory', 'warehouse_region', 'customer_segment']
numeric_cols = ['weight_kg', 'distance_km']

# Preprocess: one-hot encode categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Logistic Regression Pipeline
log_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Fit model
log_reg.fit(X_train, y_train)

# Predict and evaluate
y_pred_lr = log_reg.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print('Logistic Regression Accuracy:', round(acc_lr, 3))
print('
Classification Report (Logistic Regression):
', classification_report(y_test, y_pred_lr))

# Random Forest Pipeline
rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print('Random Forest Accuracy:', round(acc_rf, 3))
print('
Classification Report (Random Forest):
', classification_report(y_test, y_pred_rf))

# Confusion matrix for Random Forest
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Random Forest')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


## Insights and Recommendations

Based on the analysis and modeling results:

- **Shipping Modes:** Orders shipped via **Standard Class** had the highest delay rates, while **Same Day** shipments were rarely delayed. Consider shifting more critical deliveries to faster shipping modes or improving Standard Class processes.
- **Weights & Distances:** Heavier packages and those traveling longer distances experienced more delays, suggesting a need for better route planning or splitting shipments.
- **Product Categories:** Although product categories showed minor differences, certain subcategories (e.g., large furniture items) were more prone to delays due to size and handling complexity.
- **Warehouse Regions:** Delays varied by warehouse region—investigate bottlenecks in regions with higher delay rates and allocate resources accordingly.

The Random Forest model achieved higher accuracy than Logistic Regression, capturing non‑linear relationships between features. In practice, this model could be integrated into order management systems to flag high‑risk shipments so logistics managers can take proactive actions.

**Next Steps**

1. Collect more granular data (e.g., carrier performance, weather conditions) to enhance predictive power.
2. Incorporate real‑time monitoring and alerts for shipments predicted to be delayed.
3. Conduct A/B tests on operational changes (e.g., alternate carriers) to reduce delays further.

